In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=0abd80845518a9882ad546256c64960091e2f3fb27545b3ef540b2d1ea477372
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark").getOrCreate()

In [6]:
student_rdd=spark.sparkContext.textFile("/content/student.txt")
student_rdd.collect()
# compute number of students from each Institute
student_rdd.map(lambda d:((d.split(",")[1],1))).reduceByKey(lambda x,y:x+y).collect()

[('MSIS', 3), ('MIT', 4)]

In [7]:
#no of students enrolled in any program
student_rdd.map(lambda d:((d.split(",")[2],1))).reduceByKey(lambda x,y:x+y).collect()

[('Cybersecurity', 1), ('Civil', 1), ('BigData', 2), ('VLSI', 2), ('CLoud', 1)]

In [8]:
#No of boy and girl student
student_rdd.map(lambda d:(d.split(",")[3],1)).reduceByKey(lambda x,y:x+y).collect()

[('Female', 4), ('Male', 3)]

In [9]:
#Number of ‘boy’ and ‘girl’ students from selected Institute.
st=student_rdd.flatMap(lambda d:d.split("\n"))
st.filter(lambda institute:"MIT" in institute).map(lambda d:(d.split(",")[3],1)).reduceByKey(lambda x,y:x+y).collect()


[('Male', 3), ('Female', 1)]

In [10]:
city_rdd=spark.sparkContext.textFile("/content/temp.txt")
city_rdd.collect()
# city=city_rdd.flatMap(lambda d:d.split("\n"))

['23-01-2023,23.3,Udupi,India,21,55',
 '23-02-2023,25.3,Byndoor,India,25,34',
 '23-03-2023,43.3,Karkala,India,45,58',
 '23-04-2023,27.3,Udupi,India,21,55',
 '23-01-2023,23.9,Byndoor,India,21,54',
 '23-01-2023,13.9,Bangalore,India,22,54',
 '23-03-2023,34.9,Bangalore,India,22,54',
 '22-02-2023,14.9,Bangalore,India,22,54',
 '23-03-2023,24.9,Bangalore,India,22,54']

In [12]:
#Find maximum and minimum temperature of all cities from the given dataset
city_temp=city_rdd.map(lambda fields:(fields.split(",")[2],float(fields.split(",")[1])))
max=city_temp.reduceByKey(lambda x, y: x if x > y else y).collect()
min=city_temp.reduceByKey(lambda x, y: x if x < y else y).collect()
print(min)
print(max)

#city_rdd.map(lambda d:(d.split(",")[2],d.split(",")[1])).reduceByKey(lambda x,y:max(x,y)).collect()


[('Udupi', 23.3), ('Byndoor', 23.9), ('Karkala', 43.3), ('Bangalore', 13.9)]
[('Udupi', 27.3), ('Byndoor', 25.3), ('Karkala', 43.3), ('Bangalore', 34.9)]


In [13]:
#Count number of data point for each city.
city_rdd.map(lambda d:(d.split(",")[2],1)).reduceByKey(lambda x,y:x+y).collect()

[('Udupi', 2), ('Byndoor', 2), ('Karkala', 1), ('Bangalore', 4)]

In [16]:
#Find the maximum and minimum temperature for city Bangalorefrom the given dataset.
temp=city_rdd.flatMap(lambda d:d.split("\n"))
max=temp.filter(lambda city:"Bangalore" in city).map(lambda d:(d.split(",")[1])).max()
min=temp.filter(lambda city:"Bangalore" in city).map(lambda d:(d.split(",")[1])).min()
print(min)
print(max)


13.9
34.9


In [15]:
#Find the maximum and minimum temperature for any given city from the given dataset. City name should be passed through command line argument.
c=input("Enter a city:")
temp=city_rdd.flatMap(lambda d:d.split("\n"))
max=temp.filter(lambda city:c in city).map(lambda d:(d.split(",")[1])).max()
min=temp.filter(lambda city:c in city).map(lambda d:(d.split(",")[1])).min()
print(min)
print(max)


Enter a city:Bangalore
13.9
34.9


In [18]:
#Count unique number of Bank ID
bank_rdd=spark.sparkContext.textFile("/content/Bank.txt")
bank=bank_rdd.flatMap(lambda d:d.split("\n"))
bank_id=bank.map(lambda d:(d.split("\t")[0])).distinct()
bank_id.collect()

['CAN00123', 'SBI00042', 'ICI00072']

In [19]:
#Count unique number of customers

customer = bank.map(lambda d:(d.split("\t")[1])).distinct()
customer.count()


15

In [ ]:
#Number of transactions for given Account Number
bank.map(lambda d:(d.split('\t')[1],1)).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 6),
 ('123872822', 2),
 ('123876171', 2),
 ('123876142', 4),
 ('123823122', 1),
 ('123809822', 1),
 ('123376422', 1),
 ('123802671', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 5),
 ('123822322', 1),
 ('123830822', 1)]

In [20]:
#Count unique number of customer per bank id
bank.map(lambda d:(d.split("\t")[0],d.split("\t")[1])).distinct().map(lambda d:(str(d).split(',')[0],1)).reduceByKey(lambda a,b:a+b).collect()


[("('SBI00042'", 7), ("('ICI00072'", 6), ("('CAN00123'", 6)]

In [ ]:
#Number of transactions for given Account Number
bank.map(lambda d:(d.split('\t')[1],1)).reduceByKey(lambda a,b:a+b).collect()


[('123876122', 6),
 ('123872822', 2),
 ('123876171', 2),
 ('123876142', 4),
 ('123823122', 1),
 ('123809822', 1),
 ('123376422', 1),
 ('123802671', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 5),
 ('123822322', 1),
 ('123830822', 1)]

In [21]:
#Number of credit transactions for given Account Number in a given year

def checkCredit(d):
  if 'C' in d:
    return True
bank.map(lambda d:(d.split('\t')[1]+d.split('\t')[3])).filter(lambda d:checkCredit(d)).map(lambda d:(d.split('C')[0],1)).reduceByKey(lambda a,b:a+b).collect()

[('123876122', 2),
 ('123872822', 2),
 ('123823122', 1),
 ('123809822', 1),
 ('123823822', 2),
 ('123872322', 3),
 ('123872829', 6),
 ('720127541', 2),
 ('123456789', 4),
 ('123822322', 1),
 ('123830822', 1)]